In [1]:
from langchain_community.llms import HuggingFaceHub

In [2]:
from dotenv import load_dotenv
import os

from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser

load_dotenv()


True

In [3]:
# Define the prompt template
instructions = """
INSTRUCTION
You are a medical policy expert. I will provide you with the position statement for a medical policy. Each policy includes one or several medical acts ("medical_act") with a degree of necessity ("necessity_type") such as Medically Necessary, Not Medically Necessary, Investigational and Not Medically Necessary, Cosmetic, etc. The policy also includes compliance criteria that must be met.

Your task is to extract the different criteria logic schemas and output them in JSON format.

For conditions that must all be met, use the "ALL" logic operator. For conditions where any one can be met, use the "ANY" logic operator. If no specific conditions are provided, set "conditions" to null.

For example:

Input:
--Example Procedure 1--
Investigational and Not Medically Necessary:
The procedure is considered investigational and not medically necessary under the specified conditions.

Output:
[
  {{
    "medical_act": "Example Procedure 1",
    "sub_medical_act": null,
    "necessity_type": "Investigational and Not Medically Necessary",
    "description": "The procedure is considered investigational and not medically necessary under the specified conditions.",
    "conditions": null
  }}
]

Input:
--Example Procedure 2--
Medically Necessary:
<li>The procedure is considered medically necessary when all of the following criteria are met: <ol> <li>Condition A; and</li> <li>Condition B; and</li> <li>When one of the following is true: <ol start="1" style="list-style-type:lower-alpha"> <li>Condition C; or</li> <li>Condition D.</li> </ol> </li> </ol> </li>

Output:
[
  {{
    "medical_act": "Example Procedure 2",
    "sub_medical_act": null,
    "necessity_type": "Medically Necessary",
    "description": "The procedure is considered medically necessary when all of the following criteria are met:",
    "conditions": {{
      "ALL": [
        {{ "desc": "Condition A", "conditions": null }},
        {{ "desc": "Condition B", "conditions": null }},
        {{
          "ANY": [
            {{ "desc": "Condition C", "conditions": null }},
            {{ "desc": "Condition D", "conditions": null }}
          ]
        }}
      ]
    }}
  }}
]

Input:
--Example Procedure 3--
Medically Necessary:
<li>The procedure is considered medically necessary when the following criteria are met: <ol> <li>Condition X; and</li> <li>Condition Y.</li> </ol> </li>
<li>The procedure is considered medically necessary for pediatric patients when the following criteria are met: <ol> <li>Condition P; and</li> <li>Condition Q.</li> </ol> </li>

Output:
[
  {{
    "medical_act": "Example Procedure 3",
    "sub_medical_act": "Adult",
    "necessity_type": "Medically Necessary",
    "description": "The procedure is considered medically necessary when the following criteria are met:",
    "conditions": {{
      "ALL": [
        {{ "desc": "Condition X", "conditions": null }},
        {{ "desc": "Condition Y", "conditions": null }}
      ]
    }}
  }},
  {{
    "medical_act": "Example Procedure 3",
    "sub_medical_act": "Pediatric",
    "necessity_type": "Medically Necessary",
    "description": "The procedure is considered medically necessary for pediatric patients when the following criteria are met:",
    "conditions": {{
      "ALL": [
        {{ "desc": "Condition P", "conditions": null }},
        {{ "desc": "Condition Q", "conditions": null }}
      ]
    }}
  }}
]
"""

In [4]:
from huggingface_hub import HfFolder

# Load environment variables
load_dotenv()

# Get the token from environment variables
token = os.getenv("HUGGINGFACEHUB_API_TOKEN")
if not token:
    raise ValueError("Hugging Face token not found. Set the HUGGINGFACEHUB_API_TOKEN environment variable.")

# Save the token for Hugging Face API usage
HfFolder.save_token(token)

/Users/adn/Desktop/Interviews/clarity-care/claritycare/claritycare-env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [21]:
from langchain_huggingface import HuggingFacePipeline


llm = HuggingFacePipeline.from_model_id(
    model_id="mistralai/Mistral-7B-Instruct-v0.3",
    task="text-generation",
    pipeline_kwargs=dict(
        max_new_tokens=5000,
        temperature=0,
    ),
)

ValueError: Could not load the text-generation model due to missing dependencies.

In [22]:
from langchain_huggingface import HuggingFacePipeline
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer, pipeline

model_id = "neuralmagic/Mistral-7B-Instruct-v0.3-GPTQ-4bit"

# Load the custom config file
config_path = "../model/Mistral-7B-Instruct-v0.3/config.json"
config = AutoConfig.from_pretrained(config_path)

# Load the tokenizer and model with the custom config
tokenizer = AutoTokenizer.from_pretrained(model_id, config=config, use_fast=True)
model = AutoModelForCausalLM.from_pretrained(model_id, config=config)

# Create the pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=5000,
    temperature=0,
    use_auth_token=token,
)

# Create the LLM
llm = HuggingFacePipeline(pipeline=pipe)

In [25]:
# from langchain.schema import (
#     HumanMessage,
#     SystemMessage,
# )

# prompt = ChatPromptTemplate.from_messages([
#     SystemMessage(content="You're a helpful assistant"),
#     HumanMessage(
#         content="What happens when an unstoppable force meets an immovable object?"
#     ),
# ])

In [26]:
# Create a prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant that follows the following instructions {instructions}.",
        ),
        ("human", "{position_statement}"),
    ]
)

parser = JsonOutputParser()

# Create a RunnableSequence with the prompt and the model
chain = prompt | chat_model | parser 

In [27]:
# Define the position statement
position_statement = """
--Percutaneous Vertebral Disc and Vertebral Endplate Procedures--
Medically Necessary:
Intraosseous basivertebral nerve ablation (BVNA) is considered medically necessary when all of the following criteria are met:
<li>Individual is skeletally mature; and</li>
<li>Chronic unremitting low back pain of at least 6 months duration is present; and</li>
<li>Has failed to respond to at least 6 months of supervised conservative medical management (for example, exercise, nonsteroidal and/or steroidal medication [unless contraindicated], physical therapy, including passive and active treatment modalities, and activity/lifestyle modification); and</li>
<li>Diagnosis of vertebrogenic pain meeting the following criteria:	<ol style="list-style-type:lower-alpha"> <li>Documented by history and physical examination; and</li> <li>Magnetic resonance imaging (MRI)-demonstrated Modic Type 1 or 2 changes in at least one vertebral endplate, at one or more levels from L3 to S1, including the following:		<ol style="list-style-type:lower-roman"> <li>Fibrovascular bone marrow changes are present (hypointense MRI signal for Modic Type 1); or</li> <li>Fatty bone marrow changes are present (hyperintense MRI signal for Modic Type 2);<br/> and</li> </ol> </li> </ol> </li>

Investigational and Not Medically Necessary:
The following procedures are considered investigational and not medically necessary:
<li>Percutaneous intradiscal electrothermal therapy; or</li>
<li>Percutaneous intradiscal radiofrequency thermocoagulation; or</li>
<li>Intradiscal biacuplasty.</li>
"""

# Run the chain
output = chain.invoke({
    # "instructions": instructions,
    "position_statement": position_statement
})
output

TemplateError: Conversation roles must alternate user/assistant/user/assistant/...